In [1]:
import bspump
import time
import random
import bspump.file
from bspump.trigger import PeriodicTrigger
from datetime import datetime

In [2]:
class RandomNumberSource(bspump.abc.source.TriggerSource):
  async def cycle(self):
      # Roll a die and run if the result is 1
      if random.randint(1, 6) == 1:
        await self.process(random.randint(1, 100))

In [3]:
class ExampleProcessor(bspump.Processor):
     def process(self, context, event):
        current_time = datetime.now()
        timestamp = current_time.strftime('%Y-%m-%d %H:%M:%S.%f')
        # sleep a random amount of time between 1 and 100ms
        time.sleep(random.randint(1, 100) / 1000)

        event = {
            "timestamp": timestamp,
            "random_number": event,
            "squared": event * event,
            "doubled": event * 2
        }
        return event

In [7]:
ExampleProcessor.process(None, None, 3)

{'timestamp': '2023-11-20 16:10:50.114359',
 'random_number': 3,
 'squared': 9,
 'doubled': 6}

In [5]:
class ExamplePipeline(bspump.Pipeline):
    def __init__(self, app, pipeline_id):
        super().__init__(app, pipeline_id)
        self.build(
            RandomNumberSource(app, self).on(PeriodicTrigger(app, interval=0.05)),
            ExampleProcessor(app, self),
            bspump.file.FileCSVSink(app, self),
        )

In [6]:
class App(bspump.BSPumpApplication):
    def __init__(self):
        super().__init__()
        svc = self.get_service("bspump.PumpService")

        svc.add_pipeline(ExamplePipeline(self, "Pipeline"))
